In [1]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "/content/drive/MyDrive/Informer-Tensorflow/models")
sys.path.insert(1, "/content/drive/MyDrive/Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=4)
    # x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = keras.layers.Conv1D(32, 1, activation='tanh')(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(128, 32)(x)
    x = ConvLayer(64)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()
model.summary()
# del model

[TensorShape([4, 2000, 32]), TensorShape([4, 2000, 32]), TensorShape([4, 2000, 32])]
[TensorShape([4, 666, 64]), TensorShape([4, 666, 64]), TensorShape([4, 666, 64])]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(4, 2000, 1)]            0         
                                                                 
 conv1d (Conv1D)             (4, 2000, 32)             64        
                                                                 
 positional_embedding (Posit  (4, 2000, 32)            0         
 ionalEmbedding)                                                 
                                                                 
 multi_head_prob_attention (  (4, 2000, 128)           29184     
 MultiHeadProbAttention)                                         
                                                                 
 conv_layer (ConvLayer)   

In [8]:
x = tf.random.normal((4, 2000, 1))
model(x)

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 6.0431937e-08,  7.1759636e-08, -1.1137563e-07,  1.6904488e-08,
         4.9890467e-08],
       [ 8.0482756e-08, -1.6137026e-08, -6.7514243e-08,  1.2194829e-07,
        -3.4628300e-08],
       [ 8.0234450e-09, -2.2239952e-08, -5.5779505e-08, -2.3622830e-08,
         2.6762784e-08],
       [ 9.5661612e-09,  4.5414978e-08, -4.8555556e-08, -4.3752230e-08,
         3.9904030e-08]], dtype=float32)>

In [10]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [11]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [16]:
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./model_checkpoints', monitor='val_accuracy', save_best_only=True)


In [13]:


model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])


In [17]:
model.fit(x=X[0:848], y=Y[0:848], batch_size=4, epochs=1000, validation_data=(X[848:1000], Y[848:1000]),
         callbacks=[tensorboard_callback])


Epoch 1/1000
212/212 [==============================] - 12s 54ms/step - loss: 1.5686 - accuracy: 0.3314 - val_loss: 1.4843 - val_accuracy: 0.3882
Epoch 2/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.3308 - accuracy: 0.3856 - val_loss: 1.1606 - val_accuracy: 0.4605
Epoch 3/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.1787 - accuracy: 0.3892 - val_loss: 1.1362 - val_accuracy: 0.4013
Epoch 4/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.1541 - accuracy: 0.4222 - val_loss: 1.2111 - val_accuracy: 0.4145
Epoch 5/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.1723 - accuracy: 0.4175 - val_loss: 1.3076 - val_accuracy: 0.3947
Epoch 6/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.1466 - accuracy: 0.4068 - val_loss: 1.1503 - val_accuracy: 0.4013
Epoch 7/1000
212/212 [==============================] - 11s 54ms/step - loss: 1.1648 - accuracy: 0.4009 - val_loss: 1.1018 -